In [ ]:
%tensorflow_version 1.x #use tensorflow magic to use version 1.x in colab

TensorFlow 1.x selected.


In [ ]:
#imports
import pandas as pd
from keras.preprocessing import text as keras_text, sequence as keras_seq
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.utils import np_utils

from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K
from keras.callbacks import *
import tensorflow as tf

Using TensorFlow backend.


In [ ]:
train=pd.read_csv("/content/drive/My Drive/minor/english_dataset/eng23train.csv") #traning data
test=pd.read_csv("/content/drive/My Drive/minor/english_dataset/eng23test.csv")  #testing data

In [ ]:
train_X=train['text']  #training text
test_X=test['text']   #testing text

In [ ]:
embed_size = 200 # how big is each word vector
maxlen = 70 # max number of words in a question to use

In [ ]:

## Tokenize the sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(train_X)+list(test_X))
train_X = tokenizer.texts_to_sequences(train_X)
test_X = tokenizer.texts_to_sequences(test_X)
word_index = tokenizer.word_index

In [ ]:
max_features = len(word_index) # how many unique words to use (i.e num rows in embedding vector)

In [ ]:
train_X =pad_sequences(train_X, maxlen=maxlen)  #padding training text to length=70
test_X = pad_sequences(test_X, maxlen=maxlen)  #padding testing text to length=70
train_Y=train['task_2']    #training label
test_Y=test['task_2']       #testing label
train_Y=np_utils.to_categorical(train_Y)   #one-hot encoded training label
test_Y=np_utils.to_categorical(test_Y)   #one-hot encoded testing label

In [ ]:
def load_glove(word_index):
    EMBEDDING_FILE = '/content/drive/My Drive/minor/glove/glove.twitter.27B.200d.txt'   #glove file
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')[:300]
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))     #open glove embediing file
    
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.005838499,0.48782197   #embedding mean, standard deviation for intializing not found vectors
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word) #get vector for ith word
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
        else:
            embedding_vector = embeddings_index.get(word.capitalize())  #if vector for lowercase notfound then try for uppercase
            if embedding_vector is not None: 
                embedding_matrix[i] = embedding_vector
    return embedding_matrix 

In [ ]:
embedding_matrix = load_glove(word_index)  #create embedding matrix using glove

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
#imlementation of attention layer in keras in this cell
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [ ]:
def model_lstm_du(embedding_matrix):
    inp = Input(shape=(maxlen,)) #input layer
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x) #bidirectional lstm layer optimized for gpu
    x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    x = AttentionWithContext()(x)  #attention layer
    x = Dense(64, activation="relu")(x)
    outp = Dense(3, activation="sigmoid")(x) #output layer
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model

In [ ]:
def model_gru_du(embedding_matrix):
    inp = Input(shape=(maxlen,)) #input layer
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(CuDNNGRU(128, return_sequences=True))(x) #bidirectional gru layer optimized for gpu
    x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
    x = AttentionWithContext()(x)  #attention layer
    x = Dense(64, activation="relu")(x)
    outp = Dense(3, activation="sigmoid")(x) #output layer
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model

In [ ]:
model2=model_lstm_du(embedding_matrix) # create bi-lstm model with attention

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 70)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 70, 200)           2492000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 70, 256)           337920    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 70, 128)           164864    
_________________________________________________________________
attention_with_context_1 (At (None, 128)               16640     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
____________________________________________

In [ ]:
model3=model_gru_du(embedding_matrix) # create bi-gru model with attention

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 70)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 70, 200)           2492000   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 70, 256)           337920    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 70, 128)           164864    
_________________________________________________________________
attention_with_context_2 (At (None, 128)               16640     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 195 

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(train_X,train_Y, test_size=0.15, random_state=42) #splitting training and validation data

In [ ]:
cp2=ModelCheckpoint('model_lstm.hdf5',monitor='val_accuracy',verbose=1,save_best_only=True)
history2=model2.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=7, batch_size=32,callbacks=[cp2]) #training bi-lstm with attn with checkpoint 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 1921 samples, validate on 340 samples
Epoch 1/7
1921/1921 [==============================] - 11s 6ms/step - loss: 0.9794 - accuracy: 0.5429 - val_loss: 0.8920 - val_accuracy: 0.6412

Epoch 00001: val_accuracy improved from -inf to 0.64118, saving model to model_lstm.hdf5
Epoch 2/7
1921/1921 [==============================] - 5s 3ms/step - loss: 0.8177 - accuracy: 0.6471 - val_loss: 0.9127 - val_accuracy: 0.6265

Epoch 00002: val_accuracy did not improve from 0.64118
Epoch 3/7
1921/1921 [==============================] - 5s 3ms/step - loss: 0.7386 - accuracy: 0.6918 - val_loss: 0.8615 - val_accuracy: 0.6618

Epoch 00003: val_accuracy improved from 0.64118 to 0.66176, saving model to model_lstm.hdf5
Epoch 4/7
1921/1921 [==============================] - 5s 3ms/step - loss: 0.6586 - accuracy: 0.7272 - val_loss: 0.9247 - val_accuracy: 0.6206

Epoch 00004: val_accuracy did not improve fro

In [ ]:
cp3=ModelCheckpoint('model_gru.hdf5',monitor='val_accuracy',verbose=1,save_best_only=True)
history3=model3.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=7, batch_size=32,callbacks=[cp3]) #training bi-gru with attn with checkpoint

Train on 1921 samples, validate on 340 samples
Epoch 1/7
1921/1921 [==============================] - 6s 3ms/step - loss: 0.9429 - accuracy: 0.5653 - val_loss: 0.8770 - val_accuracy: 0.6324

Epoch 00001: val_accuracy improved from -inf to 0.63235, saving model to model_gru.hdf5
Epoch 2/7
1921/1921 [==============================] - 5s 3ms/step - loss: 0.7725 - accuracy: 0.6762 - val_loss: 0.8569 - val_accuracy: 0.6676

Epoch 00002: val_accuracy improved from 0.63235 to 0.66765, saving model to model_gru.hdf5
Epoch 3/7
1921/1921 [==============================] - 5s 3ms/step - loss: 0.6901 - accuracy: 0.7158 - val_loss: 0.9031 - val_accuracy: 0.6706

Epoch 00003: val_accuracy improved from 0.66765 to 0.67059, saving model to model_gru.hdf5
Epoch 4/7
1921/1921 [==============================] - 5s 3ms/step - loss: 0.5980 - accuracy: 0.7574 - val_loss: 0.8745 - val_accuracy: 0.6118

Epoch 00004: val_accuracy did not improve from 0.67059
Epoch 5/7
1921/1921 [==============================]

In [ ]:
pred_y2=model2.predict(test_X) #predicting on testing data for bi-lstm with attn
pred_y3=model3.predict(test_X) #predicting on testing data for bi-gru with attn
pred_y4=(pred_y2+pred_y3)/2    #predicting on the basis of avg of probabilities for above
pred_y2=np.argmax(pred_y2,axis=1)
pred_y3=np.argmax(pred_y3,axis=1)
pred_y4=np.argmax(pred_y4,axis=1)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print("METRICS FOR TESTING DATA")
print("BiLSTM+ATTENTION")
print(classification_report(test['task_2'],pred_y2))
print("BiGRU+ATTENTION")
print(classification_report(test['task_2'],pred_y3))
print("HYBRID By Probablity")
print(classification_report(test['task_2'],pred_y4))

METRICS FOR TESTING DATA
BiLSTM+ATTENTION
              precision    recall  f1-score   support

           0       0.61      0.76      0.68        93
           1       0.62      0.77      0.68       124
           2       0.41      0.10      0.16        71

    accuracy                           0.60       288
   macro avg       0.55      0.54      0.51       288
weighted avg       0.56      0.60      0.55       288

BiGRU+ATTENTION
              precision    recall  f1-score   support

           0       0.65      0.75      0.70        93
           1       0.62      0.81      0.70       124
           2       0.41      0.10      0.16        71

    accuracy                           0.62       288
   macro avg       0.56      0.56      0.52       288
weighted avg       0.58      0.62      0.57       288

HYBRID By Probablity
              precision    recall  f1-score   support

           0       0.64      0.77      0.70        93
           1       0.60      0.80      0.69       